# Tutorial: Creating TF Records

Last updated: 09-sep-2021 by
Cristobal Donoso

In [2]:
cd ../..

/home/users/cdonoso/astromer/ASTROMER


In [3]:
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import os
import sys,os
sys.path.append(os.path.realpath('/home/users/cdonoso/astromer/workshop/'))

from core.masking import get_padding_mask
from core.astromer import get_ASTROMER, train
from core.data  import (create_dataset, 
                        pretraining_records)

# This tutorial runs only in early mode. For the optimized code use train.py
# OTHERWISE you can comment the next ljupyine and reset the kernel before the finetuning
tf.config.run_functions_eagerly(False)

%load_ext autoreload
%autoreload 2

In [4]:
source = './data/raw_data/alcock/LCs/' # lightcurves folder
metadata = './data/raw_data/alcock/alcock_dataset.dat' # metadata file
target = './data/records/alcock' # where records will be stored

### Metadata frame sample

In [5]:
meta = pd.read_csv(metadata)
meta = meta[meta['Class'] != 'UNK']
meta = meta[meta['Class'] != 'std']
# meta = meta[meta['N'] >= 20]
meta['Band'] = tf.ones(meta.shape[0])
meta = meta.rename(columns={'Unnamed: 0':'ID', 'Path_R':'Path'})
# meta = pd.concat([frame.sample(n=100) for c, frame in meta.groupby('Class')])

2021-11-18 12:39:38.422809: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-11-18 12:39:40.130841: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22354 MB memory:  -> device: 0, name: NVIDIA RTX A5000, pci bus id: 0000:81:00.0, compute capability: 8.6
2021-11-18 12:39:40.131732: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 612 MB memory:  -> device: 1, name: NVIDIA RTX A5000, pci bus id: 0000:82:00.0, compute capability: 8.6
2021-11-18 12:39:40.133047: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:2 wit

In [6]:
meta.sample(), meta.shape

(             ID Class     rPer     bPer                 Path_B  \
 9277  3.6358.78    EC  2.84197  2.84199  ./LCs/B/3.6358.78.dat   
 
                        Path  Band  
 9277  ./LCs/R/3.6358.78.dat   1.0  ,
 (20894, 7))

### Lightcurve frame sample

In [7]:
lc_df = pd.read_csv(os.path.join(source,  
                                 meta['Path'].sample(1).values[0].split('/')[-1]),
                   delim_whitespace=False)
lc_df.head()

,mjd,mag,err
0,48998.68750,-4.969,0.084
1,49006.67188,-5.150,0.093
2,49007.65625,-4.893,0.065
3,49036.68359,-5.115,0.091
4,49037.60547,-4.936,0.066


### From .csv to .record 

In [8]:
%%time
create_dataset(meta, source, target, max_lcs_per_record=20000, 
               n_jobs=7, subsets_frac=(0.6, 0.2))

100%|█████████████████████████████████████████████| 6/6 [00:27<00:00,  4.51s/it]

CPU times: user 23.4 s, sys: 915 ms, total: 24.3 s
Wall time: 27.1 s


### Loading Data

In [14]:
batch_size = 10
max_obs = 50

In [15]:
from core.data import pretraining_records

train_batches = pretraining_records(os.path.join(target, 'train'), batch_size, max_obs=max_obs)
valid_batches = pretraining_records(os.path.join(target, 'val'), batch_size, max_obs=max_obs)
test_batches = pretraining_records(os.path.join(target, 'test'), batch_size, max_obs=max_obs)

In [16]:
sum([b['input'].shape[0] for b in train_batches])

12534

In [46]:
sum([b['input'].shape[0] for b in valid_batches]) 

32150

In [47]:
sum([b['input'].shape[0] for b in test_batches]) 

10750